In [ ]:
# https://github.com/Sposito/computer-vision-quizes/blob/main/quiz-01.ipynb

# Quiz 01

## Calculating parameters from the intrinsic matrix using the Naïve Aproach

For doing so, we will just take a picture of a rectangle, in this case, the book's cover "100 years of Solitude" from Gabriel Garcia Marquez.

This method is called Naive because we are calibrating the camera using only a few parameters, namely:
<div>
<img src="pictures/quiz01/naive_diagram.png" width="300"/>
</div>

Where dX and dY are the width and height of the book and dZ is the distance from the camera to the centre of the book. The camera was aligned to the centre of the book to the best of my capabilities, using a square, a ruler and the phone's internal accelerometer:
<br/><br/>
<div>
<img src="pictures/quiz01/alignment.jpg" width="300"/>
&nbsp;
<img src="pictures/quiz01/alignment2.jpg" width="300"/>
</div>

Once I acquired the picture, the width and height of the book were measured once again on the picture space in pixels and represented by dx and dy respectively:

<div>
<img src="pictures/quiz01/naive_calib_gimp.png" width="300"/>
</div>


$ (dX,dY,dZ) = (140mm,\ 210mm,\ 300mm), (dx, dy) = (1414px,\ 2123px) $

$ (cols, rows) = (4032,\ 3024)$

$ fx = \frac{dx}{dX}dZ = 3030px ,\ fy = \frac{dy}{dY}dZ = 3032px $

So given the intrinsic matrix and doing the mentioned substitutions with the measured values, we have:


$ M_{int} =\begin{pmatrix}
fx & 0  & \frac{cols}{2} \\
0  & fy & \frac{rows}{2}\\
0  & 0  & 1
\end{pmatrix} \Rightarrow 
\begin{pmatrix}
3030  & 0      & 2016 \\
0     & 3032  & 1512\\
0     & 0      & 1
\end{pmatrix} $


## Using MatLab Calibration Toolbox

<pre><code>
Calibration results after optimization (with uncertainties):

Focal Length:          fc = [ 2998.34967   2981.70766 ] +/- [ 181.26857   377.17289 ]
Principal point:       cc = [ 1388.25215   2005.80761 ] +/- [ 115.90141   280.28614 ]
Skew:             alpha_c = [ 0.00000 ] +/- [ 0.00000  ]   => angle of pixel axes = 90.00000 +/- 0.00000 degrees
Distortion:            kc = [ 0.18996   -1.12750   0.01047   -0.03334  0.00000 ] +/- [ 0.10398   1.00100   0.01878   0.01590  0.00000 ]
Pixel error:          err = [ 3.51625   1.73032 ]
</pre></code>

<img src="pictures/quiz01/matlab_error.png" width="300"/>
<img src="pictures/quiz01/matlab_load.png" width="300"/>
<img src="pictures/quiz01/matlab_marks.png" width="300"/>

## Python SVD and Matrix Decomposition Method

Here we will use python algebraic tools in order to calculate the intrinsic and extrinsic parameters.

### Getting Camera Points
To get the camera points, after the pictures were taken, a vector line was drawn on top of the image starting from the centre and spiraling on the model to get point in all 3 faces as we can see in:
<img src="pictures/quiz01/python_gimp_measure.png" width="400"/>

Once drawn, I exported the SVG path and the coordinates extracted from it. You can find the original SVG files at pictures/quiz01/vector_points

For the world coordinates, we do have the model's measurements since we built it after all. By counting how many cells are in each direction and multiplying by the edge length, we have 3 coordinates for each point that can be found stacked up on the $ wp $ matrix. We will have a corresponding matrix with the same height with projected coordinates for each point for each photographic measurement. We are calling it here $ cp1, cp2, cp3 $ .

In [ ]:
import numpy as np

wp = [[0, 0, 0],
      [3, 0, 0],
      [3, 0, 3],
      [0, 0, 3],
      [0, 3, 3],
      [0, 3, 0],
      [3, 3, 0]]

square_size = 30 #mm

wp = np.array(wp)
wp *= square_size

cp1 = [[1536.62, 1922.22, 0],
       [1236.71, 2071.60, 0],
       [1208.92, 1720.74, 0],
       [1530.83, 1588.73, 0],
       [1821.48, 1734.63, 0],
       [1800.64, 2091.29, 0],
       [1497.25, 2268.46, 0]]

cp2 = [[1562.71, 1688.58, 0],
       [1258.64, 1814.44, 0],
       [1224.11, 1426.82, 0],
       [1548.23, 1322.12, 0],
       [1843.40, 1426.82, 0],
       [1828.92, 1820.01, 0],
       [1517.05, 1973.72, 0]]

cp3 = [[1591.85, 2545.35, 0],
       [1421.69, 2631.44, 0],
       [1428.74, 2459.27, 0],
       [1603.43, 2379.22, 0],
       [1747.41, 2497.53, 0],
       [1728.28, 2671.72, 0],
       [1555.61, 2769.38, 0]]

cp1, cp2, cp3 = np.array(cp1), np.array(cp2), np.array(cp3)

## Stacking up the parameters matrix

Comming from the world point projection,


$ \begin{pmatrix}
    x^c  \\
    y^c  \\
    z^c
  \end{pmatrix}
= \begin{pmatrix}
fx & 0  & \frac{cols}{2} \\
0  & fy & \frac{rows}{2}\\
0  & 0  & 1
\end{pmatrix} 
\begin{pmatrix}
r_1^1  & r_1^2  & r_1^3 & T_x \\
r_2^1  & r_2^2  & r_2^3 & T_y \\
r_3^1  & r_3^2  & r_3^3 & T_z
\end{pmatrix}
\begin{pmatrix}
X^w \\
Y^w \\
Z^w \\
1
\end{pmatrix} $


From what we know, the camera and world positions we want to derivate all remaining values on that matrix multiplication.

Opening the previous values up, we will face something that looks kind of the following, containing all the information we have:
$
A = \begin{pmatrix}
X_1^w  & Y_1^w  & Z_1^w & 1 & 0 & 0 & 0 & 0 & -u_1X_1^w  & -u_1Y_1^w  & -u_1Z_w13 & -u_1   \\
0 & 0 & 0 & 0 & X_1^w  & Y_1^w  & Z_1^w & 1 &  -v_1X_1^w  & -v_1Y_1^w  & -v_1Z_1^w & -u_1   \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots &\vdots  & \vdots &\vdots  & \vdots \\
X_7^w  & Y_7^w  & Z_7^w & 1 & 0 & 0 & 0 & 0 & -u_1X_7^w  & -u_1Y_7^w  & -u_1Z_7^w & -u_7   \\
0 & 0 & 0 & 0 & X_7^w  & Y_7^w  & Z_7^w & 1 &  -v_1X_7^w  & -v_1Y_7^w  & -v_1Z_7^w & -u_7
\end{pmatrix}
$

So in the following cell we will build the matrix $ A $ :



In [ ]:
no_points = len(wp)

calib_pics = [cp1, cp2, cp3]
A = []
for cp in calib_pics:
    for i in range(1):
        X, Y, Z = wp[i][0], wp[i][1], wp[i][2]
        u = cp[i][0]
        v = cp[i][1]

        A.append([X, Y, Z, 1, 0, 0, 0, 0, -u*X, -u*Y, -u*Z, -u])
        A.append([0, 0, 0, 0, X, Y, Z, 1, -v*X, -v*Y, -v*Z, -v])

A = np.matrix(A)

We can pile up all our unknown variables as a matrix acting as a column matrix:

$
\begin{pmatrix}
p_1^1 \\
p_1^2 \\
p_1^3 \\
p_1^4 \\
p_2^1 \\
p_2^2 \\
p_2^3 \\
p_2^4 \\
p_3^1 \\
p_3^2 \\
p_3^3 \\
p_3^4 
\end{pmatrix}
$



In [ ]:
U, S, Vt = np.linalg.svd(A, full_matrices=False)

S = np.matrix(S)

Finally, this should be the moment where we manage to decompose our parameters' matrix on the product of our intrinsic and extrinsic matrixes.

$
\begin{pmatrix}
p_1^1 & p_1^2 & p_1^3 & p_1^4 \\
p_2^1 & p_2^2 & p_2^3 & p_2^4 \\
p_3^1 & p_3^2 & p_3^3 & p_3^4
\end{pmatrix} = 
\begin{pmatrix}
fx & 0  & \frac{cols}{2} \\
0  & fy & \frac{rows}{2} \\
0  & 0  & 1 
\end{pmatrix} * M ext
$

In [ ]:
import scipy.linalg as linalg

K, R = linalg.rq(S)
print(R)
T = np.diag(np.sign(np.diag(K)))
R = T * R

## Conclusion

Using the [avfoundation.avcameracalibrationdata.intrinsicmatrix](https://developer.apple.com/documentation/avfoundation/avcameracalibrationdata/2881135-intrinsicmatrix) property from iOS I found out that for that camera the expected matrix wa supposed to be $
\begin{pmatrix}
2986.5 & 0  & 1155.978 \\
0  & 2986.5 & 1560.034 \\
0  & 0  & 1
\end{pmatrix}
$ for the main camera on the iPhone 11 pro used on this assignment.

It seems that giving all Naive method wobbly nature quite close of a match, most likely linked with the imperfect alignment of the central point.

For the MatLab calibration, if anything, I would blame the error to the imperfect flatness on the model and the fact that I used a 4x5 grid to calibrate, that said numbers were not too far, but std deviation was above pixel size, what isn't good.

Unfortunately, I couldn't figure out my own calibration method before the deadline  for the assignment, but I will definitely try to find help to find out what was I missing after all.